# 导入模块

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,datetime
import re

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [18]:
# ---------- 关键设置：让长文本不截断 ----------
# 设置列的最大宽度为“无限制”（None 或 -1，确保长字符串完整显示）
pd.set_option('display.max_colwidth', None)  

# ---------- 可选设置（按需开启） ----------
# 设置最大显示行数（None 表示显示所有行，若数据行数极多，可设为具体数值）
pd.set_option('display.max_rows', 20)  

# 允许 DataFrame 横向展开显示（避免列折叠）
pd.set_option('display.expand_frame_repr', True) 
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [19]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [20]:
def select_data(Time):
    sql1 = f''' -- 订单&风控信息  近10日数据   
    SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
    ,om.status, om.user_id
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
    when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
    when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
    ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
    ,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
    ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
    ,om.user_mobile,tmu.true_name,tmu.id_card_num
    ,top.total_describes,tor.decision_result,om.cancel_reason
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
    ,cc.name as channel_name         -- 来源渠道
    ,cc.channel_type_id              -- 渠道id
    ,pa.name as activity_name        -- 活动名称
    ,om.merchant_id,om.merchant_name
    ,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
    ,om.buy_service_product,tso.status as service_status 
    ,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
    , tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
    , om.order_type, om.union_rent_tag, tojo.app_type, tprm.rent_received 已收租金 
    , tolog.status 物流状态
    ,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
    ,tp.classify_id, tpmn.name 机型
    from  db_digua_business.t_order  om
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id
    -- 备注信息合并 
    left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
    on om.id = top.order_id 
    -- 服务信息
    left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
    -- 渠道名称
    left join db_digua_business.t_channel cc on om.channel = cc.scene 
    -- 活动名称
    left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
    -- 用户信息 
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    -- 商品信息
    left join db_digua_business.t_order_details tod on om.id = tod.order_id
    -- 免押信息  
    left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
    from db_digua_business.t_order_pay t 

    where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
    on topay.order_id=om.id   and  topay.rn = 1 
    -- 商家订单转移表
    left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
    -- 京东外部订单关联表
    left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
    -- 订单物流表
    left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
    -- 租后分期表
    -- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    left join db_credit.risk risk on risk.order_id=om.id
    -- 商品表
    left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB

    left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id


    where om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
    -- and pa.type!=4
    and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -9 day )               -- 近一周数据
    -- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-10-24'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-10-30'
    -- and hour(om.create_time) >= 11
    -- and hour(om.create_time) < 
    and TIME(om.create_time) <= '{Time}'

    ;
    '''
    # .format(time)

    df_order = clean.query(sql1)
    df_order.shape
    sql3 = '''
    SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
    '''
    df_risk_examine = clean.query(sql3)
    return df_order, df_risk_examine

In [21]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)

# 数据处理

In [22]:
def process_data(df_order):
    # 复制数据
    df = df_order.copy()
    # df = df[df.type!=4]
    
    # 处理日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    print(df["下单日期"].unique())
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

    # 处理备注信息
    # df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
    def getcolor(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="颜色":
                return color_list[j]["value"]
    df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)


    def getneicun(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="内存":
                return color_list[j]["value"]

    df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
    def getfangan(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="租赁方案":
                return color_list[j]["value"]
    df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)

    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")


    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
    df.loc[:,'机型内存'] = df['机型'] +'_'+ df['内存']
    
    df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
    
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
    
    # 订单去重
    df = clean.order_drop_duplicates(df)
    
    # 定义状态
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
    df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
    
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
    
    # 保留、剔除商家数据
    df_contain = df.copy()
    df = clean.drop_merchant(df)
    # 去重订单明细导出
    # today = datetime.today() 
    # t_date = today.date().strftime('%Y-%m-%d')
    
    df_j = df[df["进件"]=="进件"]
    
    df = clean.status_node(df)
    
    # 剔除拒量订单
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    day_name_cn_mapping = {
        'Monday': '星期一',
        'Tuesday': '星期二',
        'Wednesday': '星期三',
        'Thursday': '星期四',
        'Friday': '星期五',
        'Saturday': '星期六',
        'Sunday': '星期日'
    }
    
    return df, df2

# 分组聚合

In [23]:
def group_data(df0, classify):
    df_group = df0.groupby(classify).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["进件数"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group['进件数']
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["进件数"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["进件数"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group['进件数']
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group['进件数']
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group['进件数']
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group['进件数']
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group['进件数']
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["出库"]/df_group['进件数']
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

# 合并数据

In [8]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df_order, df_risk_examine = select_data(Time='23:59:59')
df_order.shape
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(14487)

# 处理日期

In [9]:
# df[df['order_id'] == 3909094]

In [10]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
print(df['下单日期'].unique())
df["月份"]=df["下单日期"].dt.month
df['date']=df['create_time'].dt.date
df['hour']=df['create_time'].dt.hour
df.shape

<DatetimeArray>
['2025-10-30 00:00:00']
Length: 1, dtype: datetime64[ns]


(14487, 58)

In [11]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                           4634
命中蚁盾分3.0强拒                                                                 1645
命中冰鉴青云分强拒                                                                  1529
命中TD212强拒                                                                   918
命中强拒加入临时黑名单                                                                 730
                                                                           ... 
命中极信sc32007分强拒, 命中策略strategy_240801强拒                                         1
蚁盾分>=70, 命中冰鉴青云分强拒, 命中云网分且评分等级D拒绝, 命中自有模型强拒, 命中模型2508强拒                       1
命中冰鉴青云分强拒, 评分等级低于C,命中拒绝, 命中策略JDB002_212强拒, 命中模型2508组合1强拒, 命中景行分DE303强拒        1
命中冰鉴青云分强拒, 评分等级低于C,命中拒绝, 命中JDB002_202098强拒, 命中模型2508组合3强拒, 命中景行分DE303强拒       1
命中冰鉴青云分强拒, 评分等级低于C,命中拒绝, 命中模型2508组合1强拒, 命中模型2508组合2强拒, 命中景行分DE303强拒           1
Name: count, Length: 291, dtype: int64

# 处理备注信息

In [12]:
# df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()

In [13]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
# df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
# df["内存"].value_counts() 

def getfangan(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="租赁方案":
            return color_list[j]["value"]
df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)
# df["租赁方案"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
# df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['机型'] + df['内存']
# 当order_id相同时，将所有记录的total_describes 进行拼接成一个字符串并赋值给total_describes
# 第一步：当 order_id 相同时，将所有记录的 total_describes 拼接成一个字符串并赋值给备注
# 第一步：拼接 total_describes
df['total_describes'] = df['total_describes'].fillna('')
df.loc[:, '备注'] = df.groupby('order_id')['total_describes'].transform(
    lambda x: ','.join(x)
)

# 第二步：提取并重新拼接 msg 内容
df.loc[:, '备注'] = df.groupby('order_id')['备注'].transform(
    lambda x: ','.join(
        [msg for text in x for msg in 
        __import__('re').findall(r'msg=(.*?)"', text) if msg]
    )
)
df[df['order_number']=='A20251006182029112']['备注']

Series([], Name: 备注, dtype: object)

In [14]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    14118
1      369
Name: count, dtype: int64

# 渠道归属

In [15]:
df.loc[:,"归属渠道"]=df.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"],x['channel_type_id'],x['order_type']),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                   8142
搜索渠道                   3260
京东渠道                   1814
单人聊天会话中的小程序消息卡片（分享）     497
八派信息                    229
                       ... 
卡包\r\n                    2
搜索-租手机                    2
顶部搜索框的搜索结果页               1
首页十二宫格及更多                 1
九州信息                      1
Name: count, Length: 29, dtype: int64

# 订单去重

In [16]:
# 保留不去重的数据
df_not_drop = df.copy()
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


# df.drop(df[df['true_name'].isin(
#             ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
#                 , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)
df.shape

(6351, 77)

# 定义状态

In [17]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [18]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()

审核状态
前置拦截       2134
进件前取消      1934
机审强拒       1150
未进件         511
人审拒绝        267
出库          144
待审核         115
已退款          52
无法联系         28
客户取消         10
出库前风控强拒       6
Name: count, dtype: int64

# 保留商家的数据

In [19]:
df_contain = df.copy()

# 剔除商家数据

In [20]:
df = clean.drop_merchant(df)

# 去重订单明细导出

In [21]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-10-31'

# 导入服务订单

In [22]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(1632, 82)

In [23]:
df = clean.status_node(df)

# 导入台账数据

In [24]:
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")  
df_ck.shape

(40963, 18)

In [25]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)


In [26]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

# 剔除拒量订单

In [27]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

# 临时需求

## 配对信审跟进人（上一天的数据，10点）

In [28]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = query(sql_name)# df_iphone17[df_iphone17['order_id'] == 3909094]

In [29]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
df_name = df_name[df_name.信审员.isin(name_list)]

In [30]:
# df_name[df_name.order_id==4367216]

In [31]:
df_iphone17 = df2[(df2['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))&(df2['下单日期']=='2025-10-30')]# 
df_iphone17_all = df_iphone17.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
df_iphone17_all[df_iphone17_all['机审通过件']==1][['order_number', '信审员', '电审拒绝原因', '备注']].to_excel('F:\需求\总需求\iPhone17系列人审拒绝原因备注_10.29.xlsx', index=False)

In [32]:
# df_iphone17_all[df_iphone17_all['待审核'] == 1][['下单日期', 'order_number', '机型内存','归属渠道', '审核状态', '颜色', '信审员']].to_excel('F:\需求\瑞彤需求\待审核匹配信审人订单.xlsx', index=False)

df_iphone17 = df[(df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))&(df['下单日期']=='2025-10-27')&(df['待审核']==1)]# 
df_iphone17[['order_number', 'product_name', '机型内存','归属渠道', 'status2', '电审拒绝原因', '备注']]

,order_number,product_name,机型内存,归属渠道,status2,电审拒绝原因,备注


## 17PM512G租送套餐每天机审通过和出库前強拒通过订单数，9.20之后

In [33]:
df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')].product_name.unique()

array(['【首月5折】iPhone 17 全新 非监管机 顺丰包邮',
       '无溢价iPhone 17 Pro Max 首月5折 全新 非监管机 顺丰包邮',
       'iPhone 17ProMax 全新苹果新品 非监管机顺丰包邮【直播',
       'iPhone 17 Pro 无锁全新苹果 非监管机 顺丰包邮 【租物】',
       '首月5折iPhone 17 Pro 无溢价全新苹果 非监管机 顺丰包邮 ',
       '首月5折无溢价iPhone 17 Pro Max 非监管机 顺丰包邮 ', 'iPhone 17 全新 非监管机 顺丰包邮',
       'iPhone 17 Pro Max 全新苹果 无锁 非监管机 顺丰包邮 【租物】',
       '【首月5折】iPhone 17 Pro Max 全新 非监管机 顺丰包邮',
       '【首月5折】iPhone 17 Pro 全新 非监管机 顺丰包邮',
       'iPhone 17 ProMax全新苹果新品非监管机顺丰包邮【生活号',
       '首月1元 iPhone 17 Pro Max 全新无溢价无锁 非监管机 顺丰包邮',
       'iPhone 17 无锁 非监管机 全新苹果 顺丰包邮 【租物】',
       '无溢价无监管 iPhone 17 Pro 首月5折官方正品 顺丰包邮 ',
       'iPhone 17 Pro 全新苹果新品 非监管机 顺丰包邮【生活号',
       'iPhone 17 Pro 全新苹果新品 非监管机 顺丰包邮【直播】',
       '无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮',
       '首月1元 iPhone 17 Pro 无溢价全新苹果 非监管机 顺丰包邮 ',
       '首月1元无溢价iPhone 17 Pro Max 非监管机 顺丰包邮 ',
       '首月1元 iPhone 17 Pro Max  无溢价 全新 非监管机 顺丰包邮',
       'iPhone 17 非监管机 首月5折 全新国行苹果 双卡双待 顺丰包邮',
       '【首月5折】iPhone17ProMax 全新无锁 无溢价 非监管机 顺丰

In [34]:
df_iphone17PM = df[(df['机型']=='iPhone 17 Pro Max') & (df['租赁方案']=='租完即送')]
# 按下单日期与身份证号去重
df_iphone17PM.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)
df_iphone17PM.loc[:, '出库前強拒通过件'] = np.where((df_iphone17PM['出库前风控强拒']==0)&(df_iphone17PM['风控通过件']==1), 1, 0)
df_iphone17PM_group = df_iphone17PM.groupby(['下单日期']).agg({'机审通过件':'sum','风控通过件':'sum','客户取消':'sum',"无法联系":'sum','出库前风控强拒':'sum','出库前強拒通过件':'sum', '是否出库':'sum'})
df_iphone17PM_group.loc['汇总', :] = df_iphone17PM_group.select_dtypes(include=[np.number]).sum()
df_iphone17PM_group

,机审通过件,风控通过件,客户取消,无法联系,出库前风控强拒,出库前強拒通过件,是否出库
下单日期,,,,,,,
2025-10-20 00:00:00,14.0,4.0,0.0,0.0,0.0,4.0,3.0
2025-10-21 00:00:00,8.0,1.0,0.0,1.0,0.0,1.0,0.0
2025-10-22 00:00:00,5.0,2.0,0.0,1.0,0.0,2.0,0.0
2025-10-23 00:00:00,9.0,5.0,0.0,0.0,1.0,4.0,1.0
2025-10-24 00:00:00,9.0,4.0,1.0,0.0,1.0,3.0,0.0
2025-10-25 00:00:00,9.0,4.0,0.0,2.0,0.0,4.0,1.0
2025-10-26 00:00:00,8.0,3.0,0.0,0.0,0.0,3.0,3.0
2025-10-27 00:00:00,12.0,2.0,0.0,0.0,0.0,2.0,1.0
2025-10-28 00:00:00,9.0,5.0,0.0,0.0,0.0,5.0,3.0


In [35]:
df_iphone17PM['客户取消']=np.where(df_iphone17PM['status2']=='已退款', 1, df_iphone17PM['客户取消'])
df_iphone17PM[df_iphone17PM['出库前強拒通过件']==1][['order_number', 'order_id', '租赁方案', '机审通过件', '客户取消',"无法联系", '出库前风控强拒', '出库前強拒通过件', '是否出库','status2']]

,order_number,order_id,租赁方案,机审通过件,客户取消,无法联系,出库前风控强拒,出库前強拒通过件,是否出库,status2
13293,A20251020180737100,4277669,租完即送,1,1,0,0,1,0,已退款
5492,A20251020122812392,4271888,租完即送,1,0,0,0,1,1,租赁中
9102,A20251020151426307,4274493,租完即送,1,0,0,0,1,1,租赁中
12229,A20251020171003198,4276760,租完即送,1,0,0,0,1,1,租赁中
23690,A20251021143225601,4285977,租完即送,1,1,1,0,1,0,已退款
...,...,...,...,...,...,...,...,...,...,...
124818,A2025102806135123,4361174,租完即送,1,1,0,0,1,0,已退款
129781,A20251028132655340,4364842,租完即送,1,1,0,0,1,0,已退款
126612,A20251028102904329,4362498,租完即送,1,0,0,0,1,1,待收货
132582,A20251028153454649,4366854,租完即送,1,0,0,0,1,1,待收货


##  iPhone17系列人审拒绝原因汇总与明细

In [36]:
# df_iphone17[['order_id', 'total_describes']].value_counts()

In [37]:
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
dfds = df_iphone17[df_iphone17['人审拒绝']==1][['order_number', 'status2', '归属渠道', '电审拒绝原因', '备注']]
dfds

,order_number,status2,归属渠道,电审拒绝原因,备注
1834,A2025102008354994,已退款,京东渠道,"已在其他平台租赁""]","考虑,芝麻分729,1+3+碎屏+溢价费，不接受预收租金,审核不通过：已在其他平台租赁"
1860,A20251020084456118,已退款,京东渠道,"命中高风险渠道""]","短期在租多台,命中历史系统强拒,审核不通过：命中高风险渠道"
2311,A20251020094109268,已退款,其他渠道场景渠道。,"拒绝电审""]","已工作两年，月收入8K，1+3+碎屏,17521176411微信,微信未回复,审核不通过：拒绝电审"
2382,A20251020094556322,已退款,搜索渠道,"命中高风险渠道""]",审核不通过：命中高风险渠道
2516,A20251020095508416,已退款,京东渠道,"短期在租多台""]","我司在租，未还半年,审核不通过：短期在租多台"
...,...,...,...,...,...
148434,A20251029160905170,已退款,京东渠道,"验证资料与电审不符且差距大""]","不接受预收,审核不通过：验证资料与电审不符且差距大"
149596,A2025102917024768,已退款,京东渠道,"已在其他平台租赁""]","已在其他平台租赁,审核不通过：已在其他平台租赁"
150519,A20251029175704762,已退款,搜索渠道,"命中高风险地址""]","高风险地址,审核不通过：命中高风险地址"
150931,A20251029182024258,已退款,京东渠道,"第三方指导电审""]","非本人接听电话,审核不通过：第三方指导电审"


In [38]:
# dfds_g = dfds.groupby('电审拒绝原因').agg({'order_number':'count'}).rename(columns={'order_number':'数量'}).sort_values('数量',ascending=False)
# dfds_g.loc[:,'占比'] = (dfds_g['数量']/dfds_g['数量'].sum()).apply(lambda x:format(x,".2%"))
# dfds_g
# with pd.ExcelWriter('F:\需求\瑞彤需求\iPhone17系列人审拒绝原因备注汇总10.23.xlsx') as writer:
#     dfds_g.to_excel(writer, sheet_name='汇总', index=True)
#     dfds.to_excel(writer, sheet_name='明细', index=False)

# iPhone 17 全系列

## iPhone17系列总体数据

In [39]:
# df_iphone17[(df_iphone17.下单日期=='2025-10-27')&(df_iphone17['是否出库']==1)][['order_number', 'status2', '归属渠道','merchant_name', 'product_name',]]
# df[(df.order_number == 'A20251027094406258')][['order_number', 'product_name', '机型']]# &(df['product_name'].str.contains(r'iPhone 17|iPhone Air'))
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
df_iphone17[(df_iphone17['下单日期']=='2025-10-28')&(df_iphone17['是否出库']==1)][['order_number', 'status2', '归属渠道','merchant_name', 'product_name','机型']]

,order_number,status2,归属渠道,merchant_name,product_name,机型
125917,A20251028094408303,待发货,京东渠道,绿能智联科技,【首月1元】iPhone17ProMax 全新无锁 无溢价 非监管机 顺丰包邮,iPhone 17 Pro Max
137803,A20251028203913391,待发货,搜索渠道,天津众联智博物贸,首月1元无溢价iPhone 17 Pro Max 非监管机 顺丰包邮,iPhone 17 Pro Max
125020,A2025102807323183,待收货,搜索渠道,广州恒租科技,首月1元无溢价iPhone 17 Pro Max 非监管机 顺丰包邮,iPhone 17 Pro Max
126167,A2025102810040926,待收货,搜索渠道,LK雷铠科技,无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮,iPhone 17 Pro Max
126612,A20251028102904329,待收货,京东渠道,北京知呈科技,【首月1元】iPhone17ProMax 全新无锁 无溢价 非监管机 顺丰包邮,iPhone 17 Pro Max
129635,A20251028131751225,待收货,京东渠道,博润新能源科技,【首月1元】iPhone17ProMax 全新无锁 无溢价 非监管机 顺丰包邮,iPhone 17 Pro Max
130375,A2025102814034446,待收货,搜索渠道,威士利信息店,无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮,iPhone 17 Pro Max
130473,A20251028140818128,待收货,支付宝直播,海南鸿丰商业保理有限公司,无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮,iPhone 17 Pro Max
130603,A20251028141307235,待收货,搜索渠道,海南鸿丰商业保理有限公司,无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮,iPhone 17 Pro Max
131037,A20251028143311582,待收货,搜索渠道,小瓜智链科技,无溢价iPhone 17 Pro Max 首月1元 全新 非监管机 顺丰包邮,iPhone 17 Pro Max


In [40]:
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_all = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, '下单日期')
df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
df_iphone17_all_new = df_iphone17_all.reset_index()
df_iphone17_all_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-10-20 00:00:00,953.0,211.0,22.14%,260.0,27.28%,188.0,72.31%,72.0,50.0,22.0,8.46%,3.0,3.0,4.0,0.0,12.0,0.0,4.62%,4.62%,0.00%,1.15%,19.23%,1.54%,1.15%,1.26%
1,2025-10-21 00:00:00,831.0,142.0,17.09%,230.0,27.68%,186.0,80.87%,44.0,30.0,14.0,6.09%,2.0,3.0,4.0,0.0,5.0,0.0,2.17%,2.17%,0.00%,0.87%,13.04%,1.74%,1.30%,0.60%
2,2025-10-22 00:00:00,746.0,158.0,21.18%,195.0,26.14%,142.0,72.82%,53.0,32.0,21.0,10.77%,4.0,3.0,8.0,0.0,6.0,0.0,3.08%,3.08%,0.00%,2.05%,16.41%,4.10%,1.54%,0.80%
3,2025-10-23 00:00:00,765.0,155.0,20.26%,178.0,23.27%,131.0,73.60%,47.0,29.0,18.0,10.11%,4.0,5.0,4.0,0.0,5.0,0.0,2.81%,2.81%,0.00%,2.25%,16.29%,2.25%,2.81%,0.65%
4,2025-10-24 00:00:00,920.0,237.0,25.76%,187.0,20.33%,139.0,74.33%,48.0,31.0,17.0,9.09%,7.0,4.0,4.0,0.0,2.0,0.0,1.07%,1.07%,0.00%,3.74%,16.58%,2.14%,2.14%,0.22%
5,2025-10-25 00:00:00,722.0,175.0,24.24%,150.0,20.78%,117.0,78.00%,33.0,22.0,11.0,7.33%,0.0,2.0,1.0,0.0,8.0,0.0,5.33%,5.33%,0.00%,0.00%,14.67%,0.67%,1.33%,1.11%
6,2025-10-26 00:00:00,668.0,125.0,18.71%,181.0,27.10%,142.0,78.45%,39.0,20.0,19.0,10.50%,1.0,3.0,7.0,1.0,7.0,0.0,3.87%,3.87%,0.00%,0.55%,11.05%,3.87%,1.66%,1.05%
7,2025-10-27 00:00:00,857.0,192.0,22.40%,204.0,23.80%,156.0,76.47%,48.0,30.0,18.0,8.82%,9.0,3.0,1.0,0.0,5.0,0.0,2.45%,2.45%,0.00%,4.41%,14.71%,0.49%,1.47%,0.58%
8,2025-10-28 00:00:00,776.0,187.0,24.10%,192.0,24.74%,129.0,67.19%,63.0,38.0,25.0,13.02%,2.0,2.0,3.0,9.0,9.0,0.0,4.69%,4.69%,0.00%,1.04%,19.79%,1.56%,1.04%,1.16%
9,2025-10-29 00:00:00,645.0,145.0,22.48%,150.0,23.26%,114.0,76.00%,36.0,18.0,18.0,12.00%,1.0,0.0,2.0,10.0,5.0,0.0,3.33%,3.33%,0.00%,0.67%,12.00%,1.33%,0.00%,0.78%


In [41]:
# df_iphone17[(df_iphone17['是否进件']==1)&(df_iphone17['下单日期']=='2025-09-25')]['order_number'].to_excel('F:\需求\总需求\iPhone17系列订单.xlsx')
df_iphone17[(df_iphone17['下单日期']=='2025-10-20')&(df_iphone17['待审核']==1)][['order_id', 'order_number', 'status','是否拒量']]

,order_id,order_number,status,是否拒量


## iPhone17系列租物数据

In [42]:
# df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
# df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
# df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
# df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
# df_iphone17_zw_group_new

## iPhone17系列搜索数据

In [43]:
# df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
# df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
# df_iphone17_ss_group = all_models.data_group(df_iphone17_ss, df_iphone17_ss_2, df_risk_examine, '下单日期')
# df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
# df_iphone17_ss_group_new

## iPhone17系列京东数据

In [44]:
df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
# df_iphone17_jd_group = all_models.data_group(df_iphone17_jd, df_iphone17_jd_2, df_risk_examine, '下单日期')
# df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
# df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
# df_iphone17_jd_group_new.tail()
df_iphone17_jd.下单日期.unique()

<DatetimeArray>
['2025-10-20 00:00:00', '2025-10-21 00:00:00', '2025-10-22 00:00:00',
 '2025-10-23 00:00:00', '2025-10-24 00:00:00', '2025-10-25 00:00:00',
 '2025-10-26 00:00:00', '2025-10-27 00:00:00', '2025-10-28 00:00:00',
 '2025-10-29 00:00:00']
Length: 10, dtype: datetime64[ns]

### iPhone17系列京东机型转化数据

In [45]:
df_iphone17_jd_1020 = df_iphone17_jd[df_iphone17_jd['下单日期']=='2025-10-20']
df_iphone17_jd_2_1020 = df_iphone17_jd_2[df_iphone17_jd_2['下单日期']=='2025-10-20']
df_iphone17_jd_group = all_models.data_group(df_iphone17_jd_1020, df_iphone17_jd_2_1020, df_risk_examine, '机型内存')
df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
df_iphone17_jd_group_new#.tail()

,机型内存,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,iPhone 17 Pro Max256GB,53.0,14.0,26.42%,35.0,66.04%,27.0,77.14%,8.0,7.0,1.0,2.86%,0.0,0.0,0.0,0.0,1.0,0.0,2.86%,2.86%,0.00%,0.00%,20.00%,0.00%,0.00%,1.89%
1,iPhone 17 Pro Max512GB,8.0,5.0,62.50%,3.0,37.50%,2.0,66.67%,1.0,1.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,33.33%,0.00%,0.00%,0.00%
2,iPhone 17 Pro256GB,15.0,7.0,46.67%,7.0,46.67%,5.0,71.43%,2.0,0.0,2.0,28.57%,0.0,0.0,0.0,0.0,2.0,0.0,28.57%,28.57%,0.00%,0.00%,0.00%,0.00%,0.00%,13.33%
3,iPhone 17 Pro512GB,3.0,1.0,33.33%,2.0,66.67%,1.0,50.00%,1.0,0.0,1.0,50.00%,0.0,0.0,0.0,0.0,1.0,0.0,50.00%,50.00%,0.00%,0.00%,0.00%,0.00%,0.00%,33.33%
4,iPhone 17256GB,18.0,9.0,50.00%,9.0,50.00%,6.0,66.67%,3.0,3.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,33.33%,0.00%,0.00%,0.00%
5,iPhone 17512GB,3.0,0.0,0.00%,3.0,100.00%,3.0,100.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
6,合计,100.0,36.0,NaN,59.0,NaN,44.0,NaN,15.0,11.0,4.0,NaN,0.0,0.0,0.0,0.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_iphone17_jd_1021 = df_iphone17_jd[df_iphone17_jd['下单日期']=='2025-10-21']
df_iphone17_jd_2_1021 = df_iphone17_jd_2[df_iphone17_jd_2['下单日期']=='2025-10-21']
df_iphone17_jd_group_1021 = all_models.data_group(df_iphone17_jd_1021, df_iphone17_jd_2_1021, df_risk_examine, '机型内存')
df_iphone17_jd_group_1021 = df_iphone17_jd_group_1021[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_jd_group_1021.loc['合计'] = df_iphone17_jd_group_1021.select_dtypes(include=[np.number]).sum()
df_iphone17_jd_group_1021_new = df_iphone17_jd_group_1021.reset_index()
df_iphone17_jd_group_1021_new#.tail()

,机型内存,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,iPhone 17 Pro256GB,7.0,2.0,28.57%,4.0,57.14%,3.0,75.00%,1.0,1.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%,0.00%,0.00%
1,iPhone 17 Pro512GB,3.0,0.0,0.00%,3.0,100.00%,1.0,33.33%,2.0,1.0,1.0,33.33%,1.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,33.33%,33.33%,0.00%,0.00%,0.00%
2,iPhone 17256GB,7.0,3.0,42.86%,2.0,28.57%,2.0,100.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3,iPhone 17512GB,1.0,1.0,100.00%,0.0,0.00%,0.0,nan%,0.0,0.0,0.0,nan%,0.0,0.0,0.0,0.0,0.0,0.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,0.00%
4,合计,18.0,6.0,NaN,9.0,NaN,6.0,NaN,3.0,2.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# with pd.ExcelWriter('F:\需求\崔婷玉需求\iPhone17系列京东机型转化数据.xlsx') as writer:
#     df_iphone17_jd_group_new.to_excel(writer, sheet_name='10月20日', index=False)
#     df_iphone17_jd_group_1021_new.to_excel(writer, sheet_name='10月21日', index=False)

In [48]:
# # 找出重复的订单号
# duplicated_order_numbers = df_iphone17_jd[df_iphone17_jd.order_number.duplicated(keep=False)].order_number

# # 输出重复的订单号（去重显示）
# print(duplicated_order_numbers.unique())


# # df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17_2['已退款']==1)]

## iPhone17系列客户取消数据

In [49]:
# df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
# df_iphone17_qx

## iPhone17系列无法联系数据

In [50]:
# df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
# df_iphone17_wf


# iPhone 16 的遗老遗少可能有用

In [51]:

# df_16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_16 = df_16[df_16['product_name']!='iPhone 16 2024年新机 200元权益包']
# df_16['product_name'] = df_16['product_name'].str.replace(' ', '', regex=False)
# # df_16['型号内存'] = df_16['product_name'].str.split(' 国').str.get(0).str.split(' 2').str.get(0).str.replace(' ', '', regex=False).str.replace('首月5折', '', regex=False)
# df_16['型号内存'] = df_16['product_name'].str.extract(r'(iPhone ?16(ProMax|Pro|Plus)?)', expand=True)[0]

# def iPhone(df, qd_names):
#     # df_16_group = df_16.groupby(['归属渠道','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
#     df_16_group = df.groupby([f'{qd_names}','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})

#     df_16_group.rename(columns={'order_id':'去重订单数'},inplace=True )

#     df_16_group["拦截率"] = df_16_group["前置拦截"]/df_16_group["去重订单数"]
#     df_16_group["拦截率"] = df_16_group["拦截率"].apply(lambda x:format(x,".2%"))

#     df_16_group['预授权通过率'] = df_16_group["是否进件"]/df_16_group["去重订单数"]
#     df_16_group["预授权通过率"] = df_16_group["预授权通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["强拒比例"] = df_16_group["机审强拒"]/df_16_group["是否进件"]
#     df_16_group["强拒比例"] = df_16_group["强拒比例"].apply(lambda x:format(x,".2%"))

#     df_16_group["机审通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]

#     df_16_group["风控通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]-df_16_group["人审拒绝"]
#     df_16_group["风控通过率"] = df_16_group["风控通过件"]/df_16_group["是否进件"]
#     df_16_group["风控通过率"] = df_16_group["风控通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["取消率"] = df_16_group["客户取消"]/df_16_group["是否进件"]
#     df_16_group["取消率"] = df_16_group["取消率"].apply(lambda x:format(x,".2%"))


#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝"]/df_16_group["是否进件"]
#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

#     df_16_group["进件出库率"]=df_16_group["是否出库"]/df_16_group["是否进件"]
#     df_16_group["进件出库率"]=df_16_group["进件出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group["订单出库率"]=df_16_group["是否出库"]/df_16_group["去重订单数"]
#     df_16_group["订单出库率"]=df_16_group["订单出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group = df_16_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核"]]
#     df_16_group
#     return df_16_group


In [52]:
# # 计算环比
# def calculate_growth_rate(current, previous):
#     if previous == 0:
#         return float('inf') if current > 0 else float('-inf')
#     return (current - previous) / previous

In [53]:
# # 获取渠道的具体渠道
# qd_names = ['来源渠道', '归属渠道']
# indx1 = 1
# qd_group = iPhone(df_16, qd_names[indx1])
# qd_lists = ['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']
# qd_list =  qd_group.reset_index()[qd_names[indx1]][qd_group.reset_index()[qd_names[indx1]].isin(qd_lists)].drop_duplicates().to_list()


In [54]:
# # 渠道明细 去重订单数
# qd_type = ['去重订单数', '是否进件']
# indx2 = 0
# df_16_group_qcdd = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_qcdd = df_16_group_qcdd.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_qcdd = df_16_group_qcdd.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_qcdd = df_pivot_qcdd[ qd_list].reset_index()

In [55]:
# # 渠道明细 是否进件
# indx2 = 1
# df_16_group_jj = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_jj = df_16_group_jj.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_jj = df_16_group_jj.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_jj = df_pivot_jj[ qd_list].reset_index()

In [56]:
# # 渠道数据汇总
# # 提取最后两行的下单日期用于对比
# current = df_pivot_jj['下单日期'].iloc[-1].strftime('%m%d')
# previous = df_pivot_jj['下单日期'].iloc[-2].strftime('%m%d')
# df_16_qcdd = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# df_16_jj = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# for qd_name in qd_list:
#     df_16_qcdd[qd_name] =  [df_pivot_qcdd[qd_name].diff().iloc[-1],  (df_pivot_qcdd[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]
#     df_16_jj[qd_name] =  [df_pivot_jj[qd_name].diff().iloc[-1],  (df_pivot_jj[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]

# indx = 0
# df_pivot_qcdds = pd.concat([df_pivot_qcdd[indx:], df_16_qcdd])
# df_pivot_jjs = pd.concat([df_pivot_jj[indx:], df_16_jj])
# # df_pivot_jjs

# 设置定时任务

In [24]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc

# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1(Time):
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的{Time}')
    
    df_order, df_risk_examine = select_data(Time)
    # df_risk_examine = query(sql3)
    # 数据处理
    df, df2 = process_data(df_order)
    
    #iphone 17系列转化数据，总体
    df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
    # df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
    df_iphone17_all = group_data(df_iphone17, '下单日期')
    df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    # 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
    df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
    df_iphone17_all_new = df_iphone17_all.reset_index()
    
    # 租物
    df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
    # df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
    # df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
    df_iphone17_zw_group = group_data(df_iphone17_zw, '下单日期')
    df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
    
    # 搜索渠道
    df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
    # df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
    df_iphone17_ss_group = group_data(df_iphone17_ss, '下单日期')
    df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
    
    # 京东渠道
    df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
    # df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
    df_iphone17_jd_group = group_data(df_iphone17_jd, '下单日期')
    df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
    
    # 客户取消数据
    df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]

    # 无法联系数据
    df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
    
    
    with pd.ExcelWriter(path1 + f'iPhone 17系列_{Today1}.xlsx', engine='openpyxl') as writer:
        df_iphone17_all_new.to_excel(writer, sheet_name='iPhone17系列总体', index=False)
        df_iphone17_zw_group_new.to_excel(writer, sheet_name='iPhone17系列芝麻租物', index=False)
        df_iphone17_ss_group_new.to_excel(writer, sheet_name='iPhone17系列搜索渠道', index=False)
        df_iphone17_jd_group_new.to_excel(writer, sheet_name='iPhone17系列京东渠道', index=False)
        df_iphone17_qx.to_excel(writer, sheet_name='iPhone17系列取消明细', index=False)
        df_iphone17_wf.to_excel(writer, sheet_name='iPhone17系列无法联系明细', index=False)
    del df_iphone17_all_new,df_iphone17_zw_group_new,df_iphone17_ss_group_new,df_iphone17_jd_group_new,df_iphone17_qx,df_iphone17_wf
    gc.collect()
    print("iPhone 17系列数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler1 = BackgroundScheduler()
scheduler2 = BackgroundScheduler()
# 添加一个cron任务，每天的17点30分执行
hour = 17
minute = 30
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute, args=['17:30:00'])
scheduler2.add_job(my_job1, 'cron', hour=14, minute=1, args=['14:00:00'])
# scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
my_job1(Time='17:30:00')
# my_job1(Time='14:00:00')
# my_job1(Time='23:59:59')
# 启动scheduler1
scheduler1.start()
scheduler2.start()


执行定时任务：现在是2025103117的17:30:00
<DatetimeArray>
['2025-10-22 00:00:00', '2025-10-23 00:00:00', '2025-10-24 00:00:00',
 '2025-10-25 00:00:00', '2025-10-26 00:00:00', '2025-10-27 00:00:00',
 '2025-10-28 00:00:00', '2025-10-29 00:00:00', '2025-10-30 00:00:00',
 '2025-10-31 00:00:00']
Length: 10, dtype: datetime64[ns]
iPhone 17系列数据已保存,内存已回收


## iPhone 17系列昨日信审详情

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc

# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job2(Time):
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的{Time}')
    
    df_order, df_risk_examine = select_data(Time)
    # df_risk_examine = query(sql3)
    # 数据处理
    df, df2 = process_data(df_order)
    
    #iphone 17系列转化数据，总体
    df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17')]
    
    sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
    df_name = clean.query(sql_name)# df_iphone17[df_iphone17['order_id'] == 3909094]
    
    df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
    # 创建分配人映射关系 
    name_dict = {
            '小张': '李巧玲',
            '小周': '李巧凤',
            '小南': '刘三妹',
            '小何': '何静',
            '小谢': '谢金凤',
            '小咪': '杨健',
            '小慧': '林思慧',
            '小滢': '胡彩滢',
            '小星': '廖丽敏',
            '小兰': '黄兰娟',
            '小晚': '周莹',
            '小芳': '罗芳'
            }
    name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

    # 匹配映射字典
    df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
    df_name = df_name[df_name.信审员.isin(name_list)]
    
    df_iphone17 = df2[(df2['product_name'].str.contains(r'iPhone 17|iPhone Air|iPhone17'))&(df2['下单日期']=='2025-10-28')]# 
    df_iphone17_all = df_iphone17.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
    
    
    
    
    
        
    
    with pd.ExcelWriter(path1 + f'iPhone17系列人审拒绝原因备注.xlsx_{Today1}.xlsx', engine='openpyxl') as writer:
        df_iphone17_all[df_iphone17_all['机审通过件']==1][['order_number', '信审员', '电审拒绝原因', '备注']].to_excel(writer, sheet_name='人审取消原因备注',index=False)
    del df_iphone17_all
    gc.collect()
    print("iPhone 17系列数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler3 = BackgroundScheduler()
# 添加一个cron任务，每天的17点30分执行
hour = 9
minute = 59
scheduler3.add_job(my_job1, 'cron', hour=hour, minute=minute, args=['23:59:59'])
# scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
# my_job1(Time='17:30:00')
# my_job1(Time='14:00:00')
# 启动scheduler1
scheduler3.start()



In [9]:
df_order, df_risk_examine = select_data(Time='17:30:00')

## iPhone 16系列

In [74]:

# import gc
# hour = 16
# minute = 30
# # H = datetime.now().strftime('%H')
# # M = datetime.now().strftime('%M')
# def my_job2():
    
#     # print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
#     Today1 = str(datetime.now().strftime('%Y%m%d%H'))
#     print(f'执行定时任务：现在是{Today1}的{hour}点{minute}分')
    
#     df_order = query(sql1)
#     df_risk_examine = query(sql3)
#     # 数据处理
#     df, df2 = process_data(df_order)
    
#     #iphone 17系列转化数据，总体
#     df_iphone16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
#     # df_iphone16_2 = df2[df2['product_name'].str.contains(r'iPhone 16|iPhone Air')]
#     df_iphone16_all = group_data(df_iphone16, '下单日期')
#     df_iphone16_all = df_iphone16_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     # 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
#     df_iphone16_all.loc['合计'] = df_iphone16_all.select_dtypes(include=[np.number]).sum()
#     df_iphone16_all_new = df_iphone16_all.reset_index()
    
#     # 租物
#     df_iphone16_zw = df_iphone16[df_iphone16['归属渠道']=='芝麻租物']
#     # df_iphone16_zw_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='芝麻租物']
#     # df_iphone16_zw_group = all_models.data_group(df_iphone16_zw, df_iphone16_zw_2, df_risk_examine, '下单日期')
#     df_iphone16_zw_group = group_data(df_iphone16_zw, '下单日期')
#     df_iphone16_zw_group = df_iphone16_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_zw_group.loc['合计'] = df_iphone16_zw_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_zw_group_new = df_iphone16_zw_group.reset_index()
    
#     # 搜索渠道
#     df_iphone16_ss = df_iphone16[df_iphone16['归属渠道']=='搜索渠道']
#     # df_iphone16_ss_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='搜索渠道']
#     df_iphone16_ss_group = group_data(df_iphone16_ss, '下单日期')
#     df_iphone16_ss_group = df_iphone16_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_ss_group.loc['合计'] = df_iphone16_ss_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_ss_group_new = df_iphone16_ss_group.reset_index()
    
#     # 京东渠道
#     df_iphone16_jd = df_iphone16[df_iphone16['归属渠道']=='京东渠道']
#     # df_iphone16_jd_2 = df_iphone16_2[df_iphone16_2['归属渠道']=='京东渠道']
#     df_iphone16_jd_group = group_data(df_iphone16_jd, '下单日期')
#     df_iphone16_jd_group = df_iphone16_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                         "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
#     df_iphone16_jd_group.loc['合计'] = df_iphone16_jd_group.select_dtypes(include=[np.number]).sum()
#     df_iphone16_jd_group_new = df_iphone16_jd_group.reset_index()
    
#     # 客户取消数据
#     df_iphone16_qx = df_iphone16[(df_iphone16['客户取消']==1)|(df_iphone16['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]

#     # 无法联系数据
#     df_iphone16_wf = df_iphone16[(df_iphone16['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
    
    
#     with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl') as writer:
#         df_iphone16_all_new.to_excel(writer, sheet_name='iPhone16系列总体', index=False)
#         df_iphone16_zw_group_new.to_excel(writer, sheet_name='iPhone16系列芝麻租物', index=False)
#         df_iphone16_ss_group_new.to_excel(writer, sheet_name='iPhone16系列搜索渠道', index=False)
#         df_iphone16_jd_group_new.to_excel(writer, sheet_name='iPhone16系列京东渠道', index=False)
#         df_iphone16_qx.to_excel(writer, sheet_name='iPhone16系列取消明细', index=False)
#         df_iphone16_wf.to_excel(writer, sheet_name='iPhone16系列无法联系明细', index=False)
#     del df_iphone16_all_new,df_iphone16_zw_group_new,df_iphone16_ss_group_new,df_iphone16_jd_group_new,df_iphone16_qx,df_iphone16_wf
#     gc.collect()
#     print("iPhone 16系列数据已保存,内存已回收")
#     # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
#     #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

# # scheduler1 = BackgroundScheduler()
# # # 添加一个cron任务，每天的16点30分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# # scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
# my_job2()
# # 启动scheduler1



执行定时任务：现在是2025102709的16点30分
<DatetimeArray>
['2025-10-21 00:00:00', '2025-10-22 00:00:00', '2025-10-23 00:00:00',
 '2025-10-24 00:00:00', '2025-10-25 00:00:00', '2025-10-26 00:00:00',
 '2025-10-27 00:00:00']
Length: 7, dtype: datetime64[ns]
iPhone 16系列数据已保存,内存已回收


In [75]:
# df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
# df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
# # 筛选支付宝和京东的数据
# df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
# df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
# df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
# df_iphone17_group = df_iphone17_group[['进件数', '出库']]
# # 计算进件占比与出库占比
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group

In [76]:
# # iPhone16系列、iPhone17系列占比情况更新
# # 机型+内存+渠道（支付宝、京东）+进件+出库+进件占比+出库占比，
# # 每日数据更新 至17预租数据文件内。
# def myjob2():
#     minute = 35
#     path = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
#     Today = str(datetime.now().strftime('%Y%m%d%H'))
#     print(f'执行定时任务：现在是{Today}的{minute}分')
    
#     df_order = query(sql1)
#     df_risk_examine = query(sql3)
#     # 数据处理
#     df, df2 = process_data(df_order)
#     df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
#     df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
#     # 筛选支付宝和京东的数据
#     df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
#     df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
    
#     # 筛选iphone16系列数据
#     df_iphone16 = df[df.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_2 = df2[df2.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_group = all_models.data_group(df_iphone16, df_iphone16_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone16_group = df_iphone16_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件数'] / df_iphone16_group['进件数'].sum()
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库'] / df_iphone16_group['出库'].sum()
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     #iphone 17系列转化数据，总体
#     df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone17_group = df_iphone17_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     with pd.ExcelWriter(path + f'iPhone 16-17系列占比_{Today}.xlsx', engine='openpyxl') as writer:
#         df_iphone16_group.to_excel(writer, sheet_name='iPhone16系列占比', index=True)
#         df_iphone17_group.to_excel(writer, sheet_name='iPhone17系列占比', index=True)
        
#     del df_iphone16_group,df_iphone17_group
#     gc.collect()
#     print("iPhone 16、17系列占比数据已保存,内存已回收")


# myjob2()
# scheduler2 = BackgroundScheduler()
# # 添加一个cron任务，每天的17点35分执行
# scheduler2.add_job(myjob2, 'cron', hour=17, minute=35)

# # 启动scheduler1
# scheduler2.start()


In [77]:
# ip16 = pd.read_excel(r'\\digua\迪瓜租机\18.iPhone16预租数据/iPhone 16系列_2024092018.xlsx')
# ip16 = ip16.ffill()
# ip16['下单日期2'] = pd.to_datetime(ip16['下单日期']).dt.date
# ip16_group = ip16.groupby(['来源渠道', '下单日期2']).agg({'去重订单数': 'sum', '是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['来源渠道', '下单日期2'], ascending=(False, False))
# # ip16_group = ip16_group.sort_values(by=['下单日期2'], ascending=(False))
# # with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
# #         ip16_group.to_excel(writer, sheet_name='iPhone16系列转化')
# ip16_group

In [78]:

# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'去重订单数': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='去重订单数')
# df_pivot1 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
#         df_pivot1.to_excel(writer, sheet_name='iPhone16系列转化(去重订单数)')
# # ip16_group

In [79]:
# ip16['去重订单数'].sum()

In [80]:
# 
# ip16_group['去重订单数'].sum()

In [81]:
# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='是否进件')
# df_pivot2 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl', mode='a') as writer:
#         df_pivot2.to_excel(writer, sheet_name='iPhone16系列转化(进件)')

# 待审核

In [82]:
# df_dsh = df[df['待审核']==1]
# df_dsh = df_dsh[df_dsh['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_dsh = df_dsh[['order_number', 'create_time', 'product_name', '租赁方案', '内存', '颜色', '来源渠道']]
# df_dsh.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'product_name': '型号'}, inplace=True)
# df_dsh


# with pd.ExcelWriter('F:/需求/iPhone16.xlsx', engine='openpyxl') as writer:
#     df_dsh.to_excel(writer, sheet_name='待审核', index=False)